# View Campaign and Interactions

This notebook will first showcase how to select a user and view their recommendations.

Additionally it will also allow you to simulate "browsing" behavior and then get new recommendations to show how the solution adapts to the change in behavior.

In [1]:
# Imports
import boto3

import json
import numpy as np
import pandas as pd
import time
import uuid

In [14]:
# Setup and Config
# Recommendations from Event data
personalize_runtime = boto3.client('personalize-runtime')
HRNN_Campaign_ARN = "arn:aws:personalize:us-east-1:059124553121:campaign/personalize-demo-camp"

# Define User 
USER_ID = "70"

# Tracker
TRACKING_ID = '55d1df80-16ae-4f2f-a7ea-0f861952320d'

# Establish a connection to Personalize's Event Streaming
personalize_events = boto3.client(service_name='personalize-events')

In [15]:
# Interaction config
data = pd.read_csv('./ml-100k/u.data', sep='\t', names=['USER_ID', 'ITEM_ID', 'RATING', 'TIMESTAMP'])
pd.set_option('display.max_rows', 5)
data = data[data['RATING'] > 3]                # keep only movies rated 3
data = data[['USER_ID', 'ITEM_ID', 'TIMESTAMP']] # select columns that match the columns in the schema below
data

,USER_ID,ITEM_ID,TIMESTAMP
5,298,474,884182806
7,253,465,891628467
...,...,...,...
99991,676,538,892685437
99996,716,204,879795543


In [16]:
# Item Config

items = pd.read_csv('./ml-100k/u.item', sep='|', usecols=[0,1], encoding='latin-1')
items.columns = ['ITEM_ID', 'TITLE']

user_id, item_id, _ = data.sample().values[0]
item_title = items.loc[items['ITEM_ID'] == item_id].values[0][-1]
print("USER: {}".format(user_id))
print("ITEM: {}".format(item_title))

items

USER: 339
ITEM: Butch Cassidy and the Sundance Kid (1969)


,ITEM_ID,TITLE
0,2,GoldenEye (1995)
1,3,Four Rooms (1995)
...,...,...
1679,1681,You So Crazy (1994)
1680,1682,Scream of Stone (Schrei aus Stein) (1991)


In [18]:
# Get Recommendations as is
get_recommendations_response = personalize_runtime.get_recommendations(
    campaignArn = HRNN_Campaign_ARN,
    userId = str(USER_ID),
)

item_list = get_recommendations_response['itemList']
title_list = [items.loc[items['ITEM_ID'] == np.int(item['itemId'])].values[0][-1] for item in item_list]

print("Recommendations: {}".format(json.dumps(title_list, indent=2)))
print(item_list)

Recommendations: [
  "Dances with Wolves (1990)",
  "Back to the Future (1985)",
  "Dead Poets Society (1989)",
  "When Harry Met Sally... (1989)",
  "Terminator 2: Judgment Day (1991)",
  "Forrest Gump (1994)",
  "Star Wars (1977)",
  "Jaws (1975)",
  "Die Hard (1988)",
  "Star Trek: The Wrath of Khan (1982)",
  "Raiders of the Lost Ark (1981)",
  "Fugitive, The (1993)",
  "Wizard of Oz, The (1939)",
  "Speed (1994)",
  "Babe (1995)",
  "Blade Runner (1982)",
  "Schindler's List (1993)",
  "Terminator, The (1984)",
  "Monty Python and the Holy Grail (1974)",
  "Hunt for Red October, The (1990)",
  "Apollo 13 (1995)",
  "Client, The (1994)",
  "Get Shorty (1995)",
  "Gone with the Wind (1939)",
  "Stand by Me (1986)"
]
[{'itemId': '97'}, {'itemId': '204'}, {'itemId': '196'}, {'itemId': '216'}, {'itemId': '96'}, {'itemId': '69'}, {'itemId': '50'}, {'itemId': '234'}, {'itemId': '144'}, {'itemId': '228'}, {'itemId': '174'}, {'itemId': '79'}, {'itemId': '132'}, {'itemId': '568'}, {'itemId'

In [19]:
# Functions to simulate behavior

In [20]:
session_dict = {}

In [21]:
def send_movie_click(USER_ID, ITEM_ID):
    """
    Simulates a click as an envent
    to send an event to Amazon Personalize's Event Tracker
    """
    # Configure Session
    try:
        session_ID = session_dict[USER_ID]
    except:
        session_dict[USER_ID] = str(uuid.uuid1())
        session_ID = session_dict[USER_ID]
        
    # Configure Properties:
    event = {
    "itemId": USER_ID,
    }
    event_json = json.dumps(event)
        
    # Make Call
    personalize_events.put_events(
    trackingId = TRACKING_ID,
    userId= USER_ID,
    sessionId = session_ID,
    eventList = [{
        'sentAt': int(time.time()),
        'eventType': 'EVENT_TYPE',
        'properties': event_json
        }]
)

In [22]:
# Simulate behavior for a user

In [23]:
# Pick a movie, we will use ID 207 or Gattica
send_movie_click(USER_ID=USER_ID, ITEM_ID=3939)

In [24]:
# Now with Click event

In [25]:
get_recommendations_response = personalize_runtime.get_recommendations(
    campaignArn = HRNN_Campaign_ARN,
    userId = str(USER_ID),
)

item_list = get_recommendations_response['itemList']
title_list = [items.loc[items['ITEM_ID'] == np.int(item['itemId'])].values[0][-1] for item in item_list]

print("Recommendations: {}".format(json.dumps(title_list, indent=2)))
print(item_list)

Recommendations: [
  "Much Ado About Nothing (1993)",
  "Room with a View, A (1986)",
  "Heathers (1989)",
  "M*A*S*H (1970)",
  "Gandhi (1982)",
  "Dances with Wolves (1990)",
  "Sound of Music, The (1965)",
  "Stand by Me (1986)",
  "Harold and Maude (1971)",
  "Lion King, The (1994)",
  "Back to the Future (1985)",
  "Birds, The (1963)",
  "Die Hard (1988)",
  "Highlander (1986)",
  "Muriel's Wedding (1994)",
  "Breakfast at Tiffany's (1961)",
  "Get Shorty (1995)",
  "Right Stuff, The (1983)",
  "Star Wars (1977)",
  "Henry V (1989)",
  "Clueless (1995)",
  "Terminator 2: Judgment Day (1991)",
  "Snow White and the Seven Dwarfs (1937)",
  "Monty Python's Life of Brian (1979)",
  "Real Genius (1985)"
]
[{'itemId': '83'}, {'itemId': '213'}, {'itemId': '433'}, {'itemId': '211'}, {'itemId': '527'}, {'itemId': '97'}, {'itemId': '143'}, {'itemId': '655'}, {'itemId': '428'}, {'itemId': '71'}, {'itemId': '204'}, {'itemId': '443'}, {'itemId': '144'}, {'itemId': '431'}, {'itemId': '381'}, {'